# Season Massey

*coming* 

### Advantages
*coming* 

### Disadvantages
*coming* 

In [14]:

import os
import torch
import numpy as np
import pandas as pd


from copy import copy
from tqdm import tqdm
from scipy import sparse
from datetime import datetime
from dotenv import load_dotenv
from scipy.linalg import solve
from scipy.optimize import minimize
from oddsmaker.state_space import Elo
from bayes_opt import BayesianOptimization
from scipy.sparse.linalg import lsqr, spsolve
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
from scipy.sparse import csr_matrix, diags, eye, hstack
from pandas.api.types import is_datetime64_any_dtype as is_datetime

load_dotenv()

True

In [15]:
# !pip install oddsmaker

In [16]:

DATA_PATH = '../data'
def load_data():
    return pd.read_csv(os.path.join(DATA_PATH, 'testing/ncaam_sample_data.csv'))

m_data = load_data()
print(m_data.season.max())
m_data.head()

2022


,season,team_score,opp_score,is_home,numot,team_fgm,team_fga,team_fgm3,team_fga3,team_ftm,...,opp_or,opp_dr,opp_ast,opp_to,opp_stl,opp_blk,opp_pf,team_name,opp_name,date
0,2003,68,62,0,0,27,58,3,14,11,...,10,22,8,18,9,2,20,Alabama,Oklahoma,2002-11-14
1,2003,70,63,0,0,26,62,8,20,10,...,20,25,7,12,8,6,16,Memphis,Syracuse,2002-11-14
2,2003,62,68,0,0,22,53,2,10,16,...,14,24,13,23,7,1,22,Oklahoma,Alabama,2002-11-14
3,2003,63,70,0,0,24,67,6,24,9,...,15,28,16,13,4,4,18,Syracuse,Memphis,2002-11-14
4,2003,55,81,-1,0,20,46,3,11,12,...,12,24,12,9,9,3,18,E Washington,Wisconsin,2002-11-15


In [17]:

def process(data):
    
    ## date must be a datetime. rating period also acceptable as a numeric
    data['date'] = pd.to_datetime(data['date'])
    
    ## In this example, I'm interested in the difference between two stats - score and rebounds
    data['team_reb'] = data['team_or'].copy()+data['team_dr'].copy()
    data['opp_reb'] = data['opp_or'].copy()+data['opp_dr'].copy()
    data = data.copy()[['date','season','team_name','opp_name','is_home','team_score','opp_score', 'team_reb','opp_reb']]
    data['score_diff'] = data['team_score'].copy()-data['opp_score'].copy()
    data['reb_diff'] = data['team_reb'].copy()-data['opp_reb'].copy()
    
    ## long format necessary
    ## if you are not familiar with the melt function, I would recommend reading about it
    ## https://towardsdatascience.com/reshape-pandas-dataframe-with-melt-in-python-tutorial-and-visualization-29ec1450bb02
    data =data.melt(
        id_vars=['date','season','team_name','opp_name','is_home'], 
        value_vars=['score_diff','reb_diff'], 
        var_name='stat', 
        value_name='difference'
    )
    
    data['result'] = np.where(data['difference']>0, 1, 0)
    data['result'] = np.where(data['difference']==0, 0.5, data['result'].copy())
    # data = data.drop(columns=['difference'])
    data = data.sort_values(by=['date','team_name','stat']).reset_index(drop=True)
    
    return data

m_data = process(m_data)
m_data.head(10)


,date,season,team_name,opp_name,is_home,stat,difference,result
0,2002-11-14,2003,Alabama,Oklahoma,0,reb_diff,6,1.0
1,2002-11-14,2003,Alabama,Oklahoma,0,score_diff,6,1.0
2,2002-11-14,2003,Memphis,Syracuse,0,reb_diff,-2,0.0
3,2002-11-14,2003,Memphis,Syracuse,0,score_diff,7,1.0
4,2002-11-14,2003,Oklahoma,Alabama,0,reb_diff,-6,0.0
5,2002-11-14,2003,Oklahoma,Alabama,0,score_diff,-6,0.0
6,2002-11-14,2003,Syracuse,Memphis,0,reb_diff,2,1.0
7,2002-11-14,2003,Syracuse,Memphis,0,score_diff,-7,0.0
8,2002-11-15,2003,E Washington,Wisconsin,-1,reb_diff,-8,0.0
9,2002-11-15,2003,E Washington,Wisconsin,-1,score_diff,-26,0.0


In [18]:
historical = m_data.copy().loc[m_data['season']<2021].reset_index(drop=True)
holdout =  m_data.copy().loc[m_data['season']==2021].reset_index(drop=True)
future =  m_data.copy().loc[m_data['season']>2021].reset_index(drop=True)

In [33]:
### for chess they historically use a k of between 12-30. I'll start with 20 as a guess
### home field advantage differs between stats and sports. I think anything between 20-50 is a good guess

elo = Elo(m_data, k=20, hfa=20)

## use info to inspect 
# elo.info()


In [34]:
## use run_history to get historical ratings that are not leaky (i.e., don't use the current game's info to cheat)
## takes 1 second for 200,000 games, not bad
## Arpad Elo used to do these calculations by hand, poor guy
history, score = elo.run_history()
history.head()



  0%|          | 0/2674 [00:00<?, ?it/s]

c:\Users\Blake\anaconda3\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 2674/2674 [00:01<00:00, 1360.36it/s]


,date,rating_period,team_name,opp_name,is_home,hfa,stat,result,pregame_rating,pregame_opp_rating,rating_adjustment,probability
0,2002-11-14,1.0,Alabama,Oklahoma,0,0,reb_diff,1.0,1500.0,1500.0,10.0,0.5
1,2002-11-14,1.0,Alabama,Oklahoma,0,0,score_diff,1.0,1500.0,1500.0,10.0,0.5
2,2002-11-14,1.0,Memphis,Syracuse,0,0,reb_diff,0.0,1500.0,1500.0,-10.0,0.5
3,2002-11-14,1.0,Memphis,Syracuse,0,0,score_diff,1.0,1500.0,1500.0,10.0,0.5
4,2002-11-14,1.0,Oklahoma,Alabama,0,0,reb_diff,0.0,1500.0,1500.0,-10.0,0.5


In [21]:


class Optimizer:
    def __init__(self):
        pass

    def optimize(self):
        raise NotImplementedError("Subclasses must implement the optimize method.")


class MasseyOptimizer(Optimizer):
    def __init__(self, decay_type, protag_col='team', antag_col='opponent', stat_col='team_sq_score', meta_cols=['location'], min_protag_games=5):
        super().__init__()
        assert(decay_type in ['time', 'games','both'])
        self.decay_type = decay_type
        self.protag_col = protag_col
        self.antag_col = antag_col
        self.stat_col = stat_col
        self.meta_cols = meta_cols
        self.min_protag_games = min_protag_games

    def load_data(self, data, path=None):
        if path is not None:
            self.data = pd.read_csv(path)
        else:
            self.data = data
        self.preprocess_data()

    def preprocess_data(self):
        # Convert date column to datetime if needed
        if isinstance(self.data['date'].iloc[0], str):
            self.data['date'] = pd.to_datetime(self.data['date'])

        # Sort data by date
        self.data = self.data.sort_values('date').reset_index(drop=True)

        # Create a team/player list
        self.protags = list(self.data[self.protag_col].unique())
        self.antags = list(self.data[self.antag_col].unique())
        self.protags = sorted(self.protags)
        self.antags = sorted(self.antags)

        self.num_protags = len(self.protags)
        self.num_antags = len(self.antags)

        self.data['protag_idx'] = self.data[self.protag_col].apply(lambda x: self.protags.index(x))
        self.data['antag_idx'] = self.data[self.antag_col].apply(lambda x: self.antags.index(x))

        assert(len(self.data)>200), "Not enough data to optimize"

    def initialize_X(self):
        X = np.zeros((len(self.data), self.num_protags+self.num_antags+len(self.meta_cols)))
        X[np.arange(len(self.data)), self.data['protag_idx']] = 1
        X[np.arange(len(self.data)), self.data['antag_idx']+self.num_protags] = 1
        for i, col in enumerate(self.meta_cols):
            X[np.arange(len(self.data)), -1] = self.data[col]
        X = sparse.csr_matrix(X)

        return X

    def run_time_opt_scipy(self, init_points=10, n_iter=30, num_test_dates=20, num_future_days=60, max_lookback=365*3, halflife_bounds=(10, 800), l2_bounds=(1e-9, 10)):#offense_halflife_bounds=(50, 800), defense_halflife_bounds=(50,800), meta_halflife_bounds=(100,800), l2_bounds=(1e-8, 1)):

        X = self.initialize_X()
        # Select random test dates
        unique_dates = self.data['date'].unique()
        ## don't take from the first 10 or so dates
        unique_dates = sorted(unique_dates)[10:]
        
        test_dates = np.random.choice(unique_dates, size=num_test_dates, replace=False)
        num_dates = len(unique_dates)

        for test_date in test_dates:
            train_data = self.data[(self.data['date'] >= test_date - pd.Timedelta(days=max_lookback)) & (self.data['date'] < test_date)].copy()
            test_data = self.data[(self.data['date'] >= test_date) & (self.data['date'] <= test_date + pd.Timedelta(days=num_future_days))].copy()
            if len(train_data) < 50 or len(test_data) < 50:
                print("Not enough data for test date", test_date)
                test_dates = np.delete(test_dates, np.where(test_dates==test_date))

        def time_bayes_objective(halflife, l2):

            decay = np.exp(-np.log(2)/halflife)

            for i, test_date in enumerate(test_dates):

                # Filter data before the given date
                idx = self.data[(self.data['date'] >= test_date - pd.Timedelta(days=max_lookback)) & (self.data['date'] < test_date)].index.values
                test_data = self.data[(self.data['date'] >= test_date) & (self.data['date'] <= test_date + pd.Timedelta(days=num_future_days))].copy()

                dw = decay ** (test_date - self.data.loc[idx, 'date'].values).reshape(-1).astype('timedelta64[D]').astype(int)
                dw = dw/np.sum(dw)
                # Calculate the time differences in days between each game and the most recent game
                W = sparse.diags(dw)
                
                X_train = X[idx]
                q = (X_train.T @ W @ X_train).toarray()
                q += l2 * np.eye(q.shape[0]) * np.trace(q) / q.shape[0]

                y = self.data.loc[idx, self.stat_col].values
                f = X_train.T @ W @ y
                # Calculate the exponential decay weights based on the time differences and half-lives
                # solution
                b = solve(q, f, assume_a='pos')

                # Split the ratings into offense and defense ratings
                offense_ratings = b[:self.num_protags]
                defense_ratings = b[self.num_protags:self.num_protags+self.num_antags]
                
                # Create a DataFrame with the ratings
                daily_ratings = pd.DataFrame({
                    'protag': self.protags,
                    'offense_rating': offense_ratings,
                    'defense_rating': defense_ratings
                })

                test_data = test_data.merge(daily_ratings.rename(columns={'protag':'team'}), on='team', how='left')
                test_data = test_data.merge(daily_ratings.rename(columns={'protag':'opponent','offense_rating':'opp_off_rating','defense_rating':'opp_def_rating'}), on='opponent', how='left')
                # print(test_data.head())
                test_data = test_data.dropna(subset=['offense_rating','defense_rating','opp_off_rating','opp_def_rating'])

                if len(test_data) <10:
                    print("Not enough data for test date", test_date)
                    continue
                
                # Calculate the predicted scores
                linear_model = LinearRegression()
                predictions = cross_val_predict(linear_model, test_data[['offense_rating','defense_rating','opp_off_rating','opp_def_rating']], test_data[self.stat_col], cv=5)
                test_data['pred_score'] = predictions
                test_data['mse'] = (test_data[self.stat_col] - test_data['pred_score']) ** 2
                mse = test_data['mse'].mean()

            return -mse  
        
        best_decay_factor = None
        best_l2 = None
        best_mse = None

        pbounds = {'halflife': halflife_bounds, 'l2': l2_bounds}
        # pbounds = {'offense_halflife': offense_halflife_bounds, 'defense_halflife': defense_halflife_bounds, 'meta_halflife':meta_halflife_bounds, 'l2': l2_bounds}
       
        # Initialize the Bayesian Optimization object
        optimizer = BayesianOptimization(f=time_bayes_objective, pbounds=pbounds, random_state=17)

        # Perform the optimization
        optimizer.maximize(init_points=init_points, n_iter=n_iter)
        
        # Get the best parameters and correlation
        best_params = optimizer.max['params']
        print(best_params)
        best_halflife = best_params['halflife']
        best_l2 = best_params['l2']
        best_mse = -optimizer.max['target']

        return best_halflife, best_l2, best_mse

    def run_full_time_opt(self, num_samples=25, num_test_dates=20, num_future_days=60, max_lookback=365*2, halflife_bounds=(50, 800), l2_bounds=(1e-8, 1)):


        optimal_halflifes = []
        optimal_l2s = []
        best_mses = []
        for i in tqdm(range(num_samples), total=num_samples):
            best_halflife, best_l2, best_mse = self.run_time_opt_scipy(13, 32, num_test_dates, num_future_days, max_lookback, halflife_bounds, l2_bounds)

            optimal_halflifes.append(best_halflife)
            optimal_l2s.append(best_l2)
            best_mses.append(best_mse)
        return optimal_halflifes, optimal_l2s, best_mses


    def get_ratings_sparse_linear_regression(self, X, y, l2):
        # Convert the input matrix X to a sparse CSR matrix
        X_sparse = csr_matrix(X)
        
        # Solve the linear regression problem using sparse matrices
        ratings, istop, itn, r1norm = lsqr(X_sparse, y, damp=l2)[:4]
        
        # Split the ratings into offense and defense ratings
        offense_ratings = ratings[:self.num_protags]
        defense_ratings = ratings[self.num_protags:self.num_protags+self.num_antags]
        
        # Create a DataFrame with the ratings
        ratings_df = pd.DataFrame({
            'protag': self.protags,
            'offense_rating': offense_ratings,
            'defense_rating': defense_ratings
        })
        
        return ratings_df

    def get_ratings_for_dates(self, dates, halflife, l2, max_lookback=365*3):
        X = self.initialize_X()
        num_dates = len(dates)

        decay = np.exp(-np.log(2)/halflife)
        ratings = []
        # dates = [pd.to_datetime('04-09-2024')]
        for i, date in tqdm(enumerate(dates), total=num_dates):
            # Filter data before the given date
            idx = self.data[(self.data['date'] >= date - pd.Timedelta(days=max_lookback)) & (self.data['date'] < date)].index.values
            dw = decay ** (date - self.data.loc[idx, 'date'].values).reshape(-1).astype('timedelta64[D]').astype(int)
            dw = dw/np.sum(dw)
            # Calculate the time differences in days between each game and the most recent game
            W = sparse.diags(dw)
            
            X_train = X[idx]
            q = (X_train.T @ W @ X_train).toarray()
            q += l2 * np.eye(q.shape[0]) * np.trace(q) / q.shape[0]

            y = self.data.loc[idx, self.stat_col].values
            f = X_train.T @ W @ y
            # Calculate the exponential decay weights based on the time differences and half-lives
            # solution
            b = solve(q, f, assume_a='pos')

            # Split the ratings into offense and defense ratings
            offense_ratings = b[:self.num_protags]
            defense_ratings = b[self.num_protags:self.num_protags+self.num_antags]
            
            # Create a DataFrame with the ratings
            ratings_df = pd.DataFrame({
                'protag': self.protags,
                'offense_rating': offense_ratings,
                'defense_rating': defense_ratings
            })

            ratings_df['date'] = date
            ratings.append(ratings_df)

        return pd.concat(ratings)

# MO = MasseyOptimizer('time', stat_col=target_stat, meta_cols=['location'])
# MO.load_data(data)
# off_halflifes, def_halflifes, optimal_meta_halflife, l2s, corrs = MO.run_full_time_opt(num_samples=25, num_test_dates=25, num_future_days=60, max_lookback=365*3.5, offense_halflife_bounds=(10, 300), defense_halflife_bounds=(300,1200), meta_halflife_bounds=(150,650), l2_bounds=(1e-9, 10))
# halflifes, l2s, mses = MO.run_full_time_opt(num_samples=25, num_test_dates=25, num_future_days=60, max_lookback=365*3.5, halflife_bounds=(10, 800), l2_bounds=(1e-9, 10))

# ratings = MO.get_ratings_for_dates([d for d in postseason_start.values()]+[pd.to_datetime(f'04-15-{season}') for season in list(range(2017,2024))], 105, 400, 400, 0.22, max_lookback=365*2.5)




In [22]:
historical

,date,season,team_name,opp_name,is_home,stat,difference,result
0,2002-11-14,2003,Alabama,Oklahoma,0,reb_diff,6,1.0
1,2002-11-14,2003,Alabama,Oklahoma,0,score_diff,6,1.0
2,2002-11-14,2003,Memphis,Syracuse,0,reb_diff,-2,0.0
3,2002-11-14,2003,Memphis,Syracuse,0,score_diff,7,1.0
4,2002-11-14,2003,Oklahoma,Alabama,0,reb_diff,-6,0.0
...,...,...,...,...,...,...,...,...
375783,2020-03-11,2020,Washington St,Colorado,0,score_diff,14,1.0
375784,2020-03-11,2020,Weber St,CS Sacramento,0,reb_diff,-4,0.0
375785,2020-03-11,2020,Weber St,CS Sacramento,0,score_diff,-8,0.0
375786,2020-03-11,2020,Xavier,DePaul,0,reb_diff,-3,0.0


In [36]:

priors = m_data.copy().merge(history.loc[history['stat']=='score_diff'][['date','team_name','pregame_rating']], on=['date','team_name'], how='left')

priors = priors.loc[priors['season']>=2015].reset_index(drop=True)
priors['date']  = pd.to_datetime(priors['date'])
priors = priors.sort_values(by=['date','team_name']).reset_index(drop=True)
priors = priors.groupby(['season','team_name'])['pregame_rating'].first().reset_index()
priors


,season,team_name,pregame_rating
0,2015,Abilene Chr,1341.032734
1,2015,Air Force,1489.743159
2,2015,Akron,1660.792207
3,2015,Alabama,1681.363357
4,2015,Alabama A&M,1180.472092
...,...,...,...
2810,2022,Wright St,1593.434082
2811,2022,Wyoming,1498.569855
2812,2022,Xavier,1813.333810
2813,2022,Yale,1660.466730


In [39]:
test = m_data.loc[m_data['season']>=2015].copy().reset_index(drop=True)
test = test.loc[test['stat']=='score_diff'].reset_index(drop=True)
test

,date,season,team_name,opp_name,is_home,stat,difference,result
0,2014-11-14,2015,Air Force,Army,-1,score_diff,-6,0.0
1,2014-11-14,2015,Akron,UMBC,1,score_diff,17,1.0
2,2014-11-14,2015,Alabama,Towson,1,score_diff,28,1.0
3,2014-11-14,2015,Alabama A&M,Dayton,-1,score_diff,-24,0.0
4,2014-11-14,2015,Alcorn St,California,-1,score_diff,-34,0.0
...,...,...,...,...,...,...,...,...
83959,2022-04-02,2022,Kansas,Villanova,0,score_diff,16,1.0
83960,2022-04-02,2022,North Carolina,Duke,0,score_diff,4,1.0
83961,2022-04-02,2022,Villanova,Kansas,0,score_diff,-16,0.0
83962,2022-04-04,2022,Kansas,North Carolina,0,score_diff,3,1.0


In [ ]:

class SeasonMassey(Optimizer):

    def __init__(self, decay_type, data, priors, protag_col='team', antag_col='opponent', prior_train_years=3):
        super().__init__()
        assert(decay_type in ['time', 'games','both'])

        self.decay_type = decay_type
        self.data = data
        self.priors = priors
        self.protag_col = protag_col
        self.antag_col = antag_col

        data_cols = list(data.columns)
        if 'season' in data_cols:
            self.season_col = 'season'
        elif 'Season' in data_cols:
            self.season_col = 'Season'
        else:
            raise ValueError("'Season' nor 'season' column not found in data")
        # assert('season' in self.data.columns), "Season column not found in data"


    def load_data(self, data, path=None):
        if path is not None:
            self.data = pd.read_csv(path)
        else:
            self.data = data
        self.preprocess_data()

    def preprocess_data(self):
        # Convert date column to datetime if needed
        if isinstance(self.data['date'].iloc[0], str):
            
            self.data['date'] = pd.to_datetime(self.data['date'])

        # Sort data by date
        self.data = self.data.sort_values('date').reset_index(drop=True)

        # # Create a team/player list # do this by season
        # self.protags = list(self.data[self.protag_col].unique())
        # self.antags = list(self.data[self.antag_col].unique())
        # self.protags = sorted(self.protags)
        # self.antags = sorted(self.antags)

        # self.num_protags = len(self.protags)
        # self.num_antags = len(self.antags)

        # self.data['protag_idx'] = self.data[self.protag_col].apply(lambda x: self.protags.index(x))
        # self.data['antag_idx'] = self.data[self.antag_col].apply(lambda x: self.antags.index(x))

        assert(len(self.data)>200), "Not enough data to optimize"

